# U.S. Geological Survey Class GW3099
Advanced Modeling of Groundwater Flow (GW3099)\
Boise, Idaho\
September 16 - 20, 2024

![title](../../../images/ClassLocation.jpg)

## Setting up and Running Prior Monte Carlo and PEST++

In [ ]:
import os
import shutil
from pathlib import Path

import flopy as fp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyemu

### *quick note* - if you want to fiddle with options, you can change where you will write the results by setting the following variable

In [ ]:
rundir = Path("../tmprun")

### We are going to make heavy use of python and `pyemu` (basically the `flopy` of the PEST world). There are a couple more high-level (but more abstract) ways to do this that are powerful and performant. In particular, one is `PstFrom` which is discussed in this [GMDSI Tutorial](https://github.com/gmdsi/GMDSI_notebooks/tree/main/tutorials/part2_01_pstfrom_pest_setup). Also, a new advance in parameterization is in the [`pypestutils`](https://github.com/pypest/pypestutils) package we we will touch on a bit later.

### For now, though, we will work through "the hard way" to better see what we are actually doing. #showyerwork

## So, first up we need to define a location for the background files to start with, and a setup directory to copy to and work in

In [ ]:
org_d = Path("../pest_background_files/")
t_d = Path("../pest_ies_setup")

In [ ]:
if t_d.exists():
    shutil.rmtree(t_d)
shutil.copytree(org_d, t_d)

### Let's start out with pilot points for HK in the aquifer and the clay layer

### get spatial reference for the model - pilot points are based on geographic coordinates (not on layer/row/column)

More information on pilot point parameterization at this [GMDSI Tutorial](https://github.com/gmdsi/GMDSI_notebooks/tree/main/tutorials/part1_07_pilotpoints_setup) or in this [USGS Scientific Investigations Report](https://pubs.usgs.gov/sir/2010/5168/pdf/sir20105168.pdf)
![image.png](./pp.png)

In [ ]:
sim = fp.mf6.MFSimulation.load(sim_ws=str(t_d))
gwf = sim.get_model()

In [ ]:
sr = pyemu.helpers.SpatialReference.from_namfile(
    str(t_d / "at.nam"), delr=gwf.dis.delr.array, delc=gwf.dis.delc.array
)
sr

### set up variograms for K pilot points

For more background on variograms and geostatistics, check out this [GMDSI tutorial](https://github.com/gmdsi/GMDSI_notebooks/tree/main/tutorials/part0_intro_to_geostatistics)

In [ ]:
pp_aq = 3
# exponential variogram for spatially varying parameters
v_aq = pyemu.geostats.ExpVario(
    contribution=1.0,  # sill
    a=pp_aq
    * 3
    * sr.delc[
        0
    ],  # range of correlation; length units of the model. In our case 'meters'
    anisotropy=1.0,  # name says it all
    bearing=0.0,  # angle in degrees East of North corresponding to anisotropy ellipse
)

# geostatistical structure for spatially varying parameters
aq_gs = pyemu.geostats.GeoStruct(variograms=v_aq, transform="log")

In [ ]:
v_aq.plot()

In [ ]:
pp_clay = 6
# exponential variogram for spatially varying parameters
v_clay = pyemu.geostats.ExpVario(
    contribution=1.0,  # sill
    a=pp_clay
    * 5
    * sr.delc[
        0
    ],  # range of correlation; length units of the model. In our case 'meters'
    anisotropy=1.0,  # name says it all
    bearing=0.0,  # angle in degrees East of North corresponding to anisotropy ellipse
)

# geostatistical structure for spatially varying parameters
clay_gs = pyemu.geostats.GeoStruct(variograms=v_clay, transform="log")

In [ ]:
v_clay.plot()

In [ ]:
ib = gwf.dis.idomain.array[0]

### we only need pilot points for two unique K files - k_aq and k_clay

In [ ]:
k_aq = np.loadtxt(t_d / "k_aq.ref")
k_clay = np.loadtxt(t_d / "k_clay.ref")

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 4))
c = ax[0].imshow(k_aq)
plt.colorbar(c, ax=ax[0])
ax[0].set_title("aquifer")
c1 = ax[1].imshow(k_clay)
plt.colorbar(c1, ax=ax[1])
ax[1].set_title("clay");

### we can define spatial zones based on the initial K values, assuming they are grouped as homogeneous and unique values

In [ ]:
uniq_aq = np.unique(k_aq)
aq_zones_dict = dict(zip(np.arange(1, len(uniq_aq) + 1), uniq_aq))
aq_zones_dict

In [ ]:
uniq_clay = np.unique(k_clay)
clay_zones_dict = dict(zip(np.arange(1, len(uniq_clay) + 1), uniq_clay))
clay_zones_dict

### make integer arrays of zones for the unique values of the aquifer ... 

In [ ]:
aq_zones = np.zeros_like(k_aq)
for ck, cv in aq_zones_dict.items():
    aq_zones[k_aq == cv] = ck

### ... and the clay

In [ ]:
# only put pilot points in the clay where the clay is actually present
clay_zones = np.zeros_like(k_aq)
clay_zones[k_clay == clay_zones_dict[1]] = 1

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(8, 6))
ax = ax.ravel()
c = ax[0].imshow(k_aq)
plt.colorbar(c, ax=ax[0])
ax[0].set_title("K aquifer")
c1 = ax[1].imshow(k_clay)
plt.colorbar(c1, ax=ax[1])
ax[1].set_title("K clay")

c2 = ax[2].imshow(aq_zones)
plt.colorbar(c2, ax=ax[2])
ax[2].set_title("Zones aquifer")

c3 = ax[3].imshow(clay_zones)
plt.colorbar(c3, ax=ax[3])
ax[3].set_title("Zones clay");

## let's set up pilot points for the k files 

In [ ]:
aq_pp_df = pyemu.pp_utils.setup_pilotpoints_grid(
    sr=sr,  # model spatial reference
    ibound=aq_zones,  # to which cells to setup ppoints
    # ibound=np.ones_like(aq_zones),
    prefix_dict={0: ["hk_aq"]},  # prefix to add to parameter names
    pp_dir=t_d,
    tpl_dir=t_d,
    shapename=str(t_d / "pp_aq.shp"),
    use_ibound_zones=True,
    every_n_cell=pp_aq,
)  # pilot point spacing

In [ ]:
plt.scatter(x=aq_pp_df.x, y=aq_pp_df.y, c=aq_pp_df.zone)
plt.axis("square")
plt.xlim([0, 12500])

In [ ]:
clay_pp_df = pyemu.pp_utils.setup_pilotpoints_grid(
    sr=sr,  # model spatial reference
    ibound=clay_zones,  # to which cells to setup ppoints
    prefix_dict={0: ["hk_clay"]},  # prefix to add to parameter names
    pp_dir=t_d,
    tpl_dir=t_d,
    shapename=str(t_d / "pp_clay.shp"),
    use_ibound_zones=True,
    every_n_cell=pp_clay,
)  # pilot point spacing

In [ ]:
plt.scatter(x=clay_pp_df.x, y=clay_pp_df.y, c=clay_pp_df.zone)
plt.axis("square")
plt.ylim([0, 20000])
plt.xlim([0, 12500])

### Now we make an `OrdinaryKrige` object for each pilot point network. This holds the information needed for setting up interpolation, and also has methods to perform the interpolation.

In [ ]:
ok_aq = pyemu.geostats.OrdinaryKrige(
    aq_gs, aq_pp_df.loc[aq_pp_df.pargp == "hk_aq"]
)
ok_clay = pyemu.geostats.OrdinaryKrige(
    clay_gs, clay_pp_df.loc[clay_pp_df.pargp == "hk_clay"]
)

### We need factors for the interpolation here. Basically a set of weights for each grid cell, based on distance from the nearest pilot points, from which a weighted average of pilot point values is used to calculate the value in each cell. The function of weight by distance is informed by the variogram above. 

In [ ]:
aq_fac_df = ok_aq.calc_factors_grid(
    sr,
    minpts_interp=1,
    maxpts_interp=10,
)
clay_fac_df = ok_clay.calc_factors_grid(
    sr,
    minpts_interp=1,
    maxpts_interp=10,
)

In [ ]:
# just gotta save these factors out to a file
ok_aq.to_grid_factors_file(str(t_d / "pp_aq.fac"))
ok_clay.to_grid_factors_file(str(t_d / "pp_clay.fac"))

In [ ]:
list(t_d.glob("*.fac"))

In [ ]:
# let's check out all the useful information we have in factors dataframe
aq_fac_df

# now make a PST file to pull this all together
### recall the general structure of the PST interface

In [ ]:
from IPython.display import Image, display

display(Image(url="../background/pest_flow1.gif"))

### so, let's have a look at the interface files (`tpl` and `ins`)

In [ ]:
list(t_d.glob("*.tpl"))

### So we have two `pp` related `tpl` files, and some homebrewed other ones made ahead of time. Let's check those out as well.

In [ ]:
# here's the river file - looks like we gonna
# parameterize conductance as homog (is that cool???)
[print(i.strip()) for i in open(t_d / "riv.ref.tpl", "r").readlines()];

In [ ]:
# rch is easy - just a constant value
[print(i.strip()) for i in open(t_d / "at.rch.tpl", "r").readlines()];

In [ ]:
# npf has some shenanigans going on
[print(i.strip()) for i in open(t_d / "at.npf.tpl", "r").readlines()];

#### for `npf`, we have just the two files we assume are being made my the pilot point interpolation - `k_aq.ref` for the aquifer and `k_clay.ref` for the clay. These two files get assigned across the 5 layers based on lithology. Not a bad idea! Then the `FACTOR` capability is used to assign a fraction to anisotropy for `k33` (queue Alden's dance). 

#### how about `ins` files?

In [ ]:
list(t_d.glob("*.ins"))

In [ ]:
# just the one we made in the previous notebook
[print(i.strip()) for i in open(t_d / "allobs.dat.ins", "r").readlines()];

#### OK! Given all this info, the method below (`Pst.from_io_files`) reads all the `tpl` and `ins` files in a directory, scrapes the parameter and observation information from them, and populated a `Pst` object in memory that contains defaults and specifics covering basically everything the PEST interface will, ultimately, need.

In [ ]:
cwd = Path(os.getcwd())
os.chdir(t_d)
pst = pyemu.Pst.from_io_files(*pyemu.helpers.parse_dir_for_io_files("."))
os.chdir(cwd)

### Both the `parameter_data` and `observation_data` are populated as `pandas DataFrames` (i know, easier if `recarrays` but....) so you can view and change values easily

In [ ]:
pars = pst.parameter_data
pars

### We need to set meaningful starting values

### first set constant parameter values

In [ ]:
pars.loc[pars.parnme.str.contains("aniso")]

In [ ]:
pars.loc["rv1", "parval1"] = 1e05  # starting value
pars.loc["rv1", "parlbnd"] = 1e03  # lower bound
pars.loc["rv1", "parubnd"] = 1e06  # upper bound
pars.loc["rv1", "pargp"] = "riv_cond"  # group (same for those below)

pars.loc["rch", "parval1"] = 0.003641
pars.loc["rch", "parlbnd"] = 0.003641 * 0.9
pars.loc["rch", "parubnd"] = 0.003641 * 1.1
pars.loc["rch", "pargp"] = "rch"

pars.loc["kaniso", "parval1"] = 0.2
pars.loc["kaniso", "parlbnd"] = 0.001
pars.loc["kaniso", "parubnd"] = 0.85
pars.loc["kaniso", "pargp"] = "anisotropy"

pars.loc["kaniso_clay", "parval1"] = 0.5
pars.loc["kaniso_clay", "parlbnd"] = 0.001
pars.loc["kaniso_clay", "parubnd"] = 0.85
pars.loc["kaniso_clay", "pargp"] = "anisotropy_clay"

### next let's use the "zone" information from the pilot points to assign groups by array _and_ zone

In [ ]:
pars.loc[~pars.zone.isnull(), "pargp"] = [
    f"{i.split('_i:')[0]}_{int(float(z))}"
    for i, z in zip(
        pars.loc[~pars.zone.isnull()].parnme,
        pars.loc[~pars.zone.isnull()].zone,
    )
]

In [ ]:
pars.zone = [float(i) for i in pars.zone]
pars.sample(5)

### Without enforcing too much structure, we can adjust a few starting values and bounds to inform ies of at least some of the general patterns of K values

In [ ]:
pars.loc[pars.pargp.str.contains("hk_aq"), "parval1"] = 200
pars.loc[pars.pargp.str.contains("hk_aq"), "parlbnd"] = 0.01
pars.loc[pars.pargp.str.contains("hk_aq"), "parubnd"] = 1e3
pars.loc[pars.pargp.str.contains("hk_clay"), "parval1"] = 1
pars.loc[pars.pargp.str.contains("hk_clay"), "parlbnd"] = 0.0001
pars.loc[pars.pargp.str.contains("hk_clay"), "parubnd"] = 1e2
pars.loc[pars.pargp == "hk_aq_5", "parval1"] = 200
pars.loc[pars.pargp == "hk_aq_4", "parval1"] = 200
pars.loc[pars.pargp == "hk_aq_3", "parval1"] = 200
pars.loc[pars.pargp == "hk_aq_4", "parubnd"] = 750
pars.loc[pars.pargp == "hk_aq_3", "parubnd"] = 500
pars.loc[pars.pargp == "hk_clay_1", "parval1"] = 1
pars.loc[pars.pargp == "hk_clay_2", "parval1"] = 150
pars.loc[pars.pargp == "hk_clay_2", "parlbnd"] = 1
pars.loc[pars.pargp == "hk_clay_2", "parubnd"] = 1000

## next we need to read in the observation values (the known values that is) and set them

In [ ]:
obsvals = pd.read_csv(t_d / "obsvalues.dat", sep=r"\s+", index_col=0)
obsvals.sample(5)

In [ ]:
pst.observation_data.sample(5)

In [ ]:
pst.observation_data["standard_deviation"] = -99999

In [ ]:
pst.observation_data.loc[obsvals.index, "obgnme"] = obsvals.obgnme
pst.observation_data.loc[obsvals.index, "obsval"] = obsvals.obsval
pst.observation_data.loc[obsvals.index, "weight"] = obsvals.weight

pst.observation_data

### we need to set standard deviation values for the observations to account for their uncertainty. This uncertainty stems from multiple sources including measurement error, the process of assigning a single value to represent more than an infinitesimal snapshot in space and time, model imperfections, etc. etc. It may seem like we don't know anything!

In [ ]:
# for heads, let's go with 0.25 m
pst.observation_data.loc[
    obsvals.obgnme == "headgroup", "standard_deviation"
] = 0.25
# for head differences, this can be lower as the values are quite a bit lower as well
pst.observation_data.loc[
    obsvals.obgnme == "headdiffgroup", "standard_deviation"
] = 0.025
# for the river flux and lake flux observations, we can assume 10% error around the observed values
pst.observation_data.loc[
    ~obsvals.obgnme.str.contains("head"), "standard_deviation"
] = 0.1 * np.abs(
    pst.observation_data.loc[~obsvals.obgnme.str.contains("head"), "obsval"]
)

In [ ]:
pst.observation_data

### now we need to write out the `forward_run` script that includes pilot point interpolation, running MF6, and postprocesses the observations to prepare them for reading

#### this is hella meta - writing python using python. careful with quote marks!

In [ ]:
with open(t_d / "forward_run.py", "w") as f:
    # add imports
    f.write(
        "import os\nimport shutil\nimport numpy as np\nimport pyemu\nimport flopy\n"
    )
    f.write("import pandas as pd\n")
    # preprocess pilot points to grid
    f.write(
        "_ = pyemu.geostats.fac2real('hk_aqpp.dat', factors_file='pp_aq.fac',out_file='k_aq.ref')\n"
    )
    f.write(
        "_ = pyemu.geostats.fac2real('hk_claypp.dat', factors_file='pp_clay.fac',out_file='k_clay.ref')\n"
    )
    # run MF6
    f.write("pyemu.os_utils.run('mf6')\n")
    # process the observations
    # remember to sort the glob results!
    outfiles = [
        f"./{i.name}"
        for i in sorted(list(Path("../pest_obs_prototype").glob("*.csv")))
    ]
    f.write(f"outfiles = {outfiles}\n")
    f.write("obs = pd.concat([pd.read_csv(i).T.iloc[1:] for i in outfiles])\n")
    f.write(
        "obs.loc['DS'] = obs.loc['DS'] + obs.loc['PF']\n"
    )  # note that we have to add the two river obs
    f.write(
        "obs.loc['UW02'] = obs.loc['U02']-obs.loc['W02']\n"
    )  # head difference targets
    f.write(
        "obs.loc['UW08'] = obs.loc['U08']-obs.loc['W08']\n"
    )  # head difference targets
    f.write(
        "obs.loc['UW15'] = obs.loc['U15']-obs.loc['W15']\n"
    )  # head difference targets
    f.write("obs.columns=['obsname']\n")
    f.write('obs.to_csv("allobs.dat", sep = " ")')

In [ ]:
# so how does it look?
[print(i.strip()) for i in open(t_d / "forward_run.py", "r").readlines()];

### now set a few ies-specific values and write out the file

In [ ]:
pst.model_command = ["python forward_run.py"]
pst.control_data.noptmax = 0  # --> we need to run once for weight rebalancing

pst.pestpp_options["ies_num_reals"] = 150
pst.pestpp_options["par_sigma_range"] = 6
pst.pestpp_options["ies_no_noise"] = "false"
pst.write(str(t_d / "mv.ies.pst"), version=2)

### copy the run folder over to be a master for parallel runs

In [ ]:
if rundir.exists():
    shutil.rmtree(rundir)
shutil.copytree(t_d, rundir)

## now we need to balance the objective function - we can start based on the initial run

### run the model once through PEST

In [ ]:
pyemu.os_utils.run("pestpp-ies mv.ies.pst", str(rundir))

### now read in the residuals and see how things look

In [ ]:
pst_new = pyemu.Pst(str(rundir / "mv.ies.pst"))

In [ ]:
pst_new.plot(kind="phi_pie")

### what are the components of the objective function?

In [ ]:
pst_new.phi_components

In [ ]:
phi = pst_new.phi
phi

In [ ]:
# REFERENCE BALANCE
# new_components = {
#     "headdiffgroup": phi * 0.15,
#     "headgroup": phi * 0.225,
#     "lakegroup": phi * 0.225,
#     "rivgroup": phi * 0.4,
# }

In [ ]:
new_components = {
    "headdiffgroup": phi * 0.15,
    "headgroup": phi * 0.225,
    "lakegroup": phi * 0.225,
    "rivgroup": phi * 0.4,
}

In [ ]:
pst_new.adjust_weights(obsgrp_dict=new_components)

In [ ]:
pst_new.phi_components

In [ ]:
pst_new.plot(kind="phi_pie")

In [ ]:
pst_new.observation_data

In [ ]:
pst_new.control_data.noptmax = 3

In [ ]:
pst_new.write(str(rundir / "mv.ies.pst"), version=2)
pst_new.write(str(t_d / "mv.ies.pst"), version=2)

### now run the parameter estimation in parallel

In [ ]:
run_flag = True
if run_flag:
    pyemu.os_utils.start_workers(
        str(t_d),
        num_workers=15,
        master_dir=str(rundir),
        exe_rel_path="pestpp-ies",
        pst_rel_path="mv.ies.pst",
    )